# Format Conversion - XML Example

> [Return to Examples List](README.md)

This example shows how to convert a TRIOS JSON file to XML, which is useful for applications that require XML format for data interchange or archival.

## Load Experiment Data
Here we load a JSON file representing experimental data. This step is crucial as it initializes the data structure that we will convert into XML.

In [1]:
# This is just to ensure you have the data file, downloaded from TA's website
from file_downloader import download_files

filename = "DSC - Epoxy - 10Cmin.json"
file_path = download_files(filename, "files/DSC kinetics/")[0]

In [2]:
from tadatakit.classes import Experiment
experiment = Experiment.from_json(file_path)

## Conversion to XML

The following function `to_xml` converts the loaded experiment data to an XML format. The function includes utilities to sanitize XML tags and properly format nested data structures, ensuring valid and well-formed XML output. This function is then added to the Experiment class with `setattr`.

In [3]:
import xml.etree.ElementTree as ET
import os
import re
from typing import Union, TextIO

def to_xml(self, path_or_file: Union[str, os.PathLike, TextIO]) -> None:
    """
    Serializes the instance to an XML file or stream.
    """
    def sanitize_key(key: str) -> str:
        """
        Sanitizes the key to make it a valid XML tag name.
        Replaces all characters except ASCII alphanumerics and underscores with underscores,
        and ensures it's a valid XML name.
        """
        # Replace all non-ASCII alphanumerics and non-underscore characters with underscores
        key = re.sub(r'[^a-zA-Z0-9_]', '_', key)
    
        # Ensure it doesn't start with a number or punctuation (except underscore)
        if re.match(r'^[0-9]', key):
            key = '_' + key
        
        # Prevent tags from starting with 'xml' in any case
        if key.lower().startswith('xml'):
            key = '_' + key
    
        return key
    
    def dict_to_xml(data, root_tag='Data'):
        def build_element(key, value):
            key = sanitize_key(key)
            element = ET.Element(key)
            if isinstance(value, dict):
                for sub_key, sub_value in value.items():
                    element.append(build_element(sub_key, sub_value))
            elif isinstance(value, list):
                for item in value:
                    item_tag = sanitize_key(key.rstrip('s'))  # Attempt to create singular form of the key
                    if isinstance(item, dict):
                        element.append(build_element(item_tag, item))
                    else:
                        sub_element = ET.Element(item_tag)
                        sub_element.text = str(item)
                        sub_element.set('type', type(item).__name__)
                        element.append(sub_element)
            else:
                element.text = str(value)
                element.set('type', type(value).__name__)
            return element

        root = ET.Element(sanitize_key(root_tag))
        for key, value in data.items():
            root.append(build_element(key, value))
        xml_string = ET.tostring(root, encoding='unicode', method='xml')
        return f'<?xml version="1.0" encoding="UTF-8"?>{xml_string}'

    xml_output = dict_to_xml(experiment.to_dict())

    try:
        if isinstance(path_or_file, (str, os.PathLike)):
            with open(path_or_file, "w", encoding='utf-8') as file:
                file.write(xml_output)
        else:
            path_or_file.write(xml_output)
    except OSError as e:
        raise IOError(f"An error occurred while writing the file: {e}")



setattr(Experiment, "to_xml", to_xml)

## Execute Conversion

With the `to_xml` method now added to our `Experiment` class, we execute the method to convert our experiment data to an XML file named '10Cmin.xml'. This XML file can be used in other applications that require XML formatted data.

In [4]:
experiment.to_xml("10Cmin.xml")

## File Contents

Rather than include the file here, you can get a preview with the first 10000 characters below.

In [5]:
with open("10Cmin.xml") as f:
    print(f.read(10000))

<?xml version="1.0" encoding="UTF-8"?><Data><Schema><Url type="str">https://software.tainstruments.com/schemas/TRIOSJSONExportSchema</Url></Schema><Operators><Operator><Name type="str">TA Instruments Expert</Name></Operator></Operators><Sample><Name type="str">DSC - Epoxy - 10Cmin</Name><Notes type="NoneType">None</Notes><PanNumber type="int">20</PanNumber><PanMass><Value type="float">53.180999755859375</Value><Unit><Name type="str">mg</Name></Unit></PanMass><Mass><Value type="float">5.508999824523926</Value><Unit><Name type="str">mg</Name></Unit></Mass><PanType type="str">Tzero Aluminum Pin Hole Hermetic</PanType><Shape type="NoneType">None</Shape><Density type="NoneType">None</Density><PoissonsRatio type="NoneType">None</PoissonsRatio></Sample><Procedure><Configuration><InstrumentConfigurations><InstrumentConfiguration><Type type="str">DSC2500</Type><Name type="str">DSC2A-00717</Name><SerialNumber type="str">DSC2A-00717</SerialNumber><Location type="str">Training Lab</Location><Refer